# Kappa distribution parameters L-Moments

This fonction helps estimates the parameters of the Kappa distribution using the L-Moments ajustement method.
There is analytical solution to the equations, so we implemented the Newton-Raphson method to solve the equations.

The original code is available in fortran  
[here](http://lib.stat.cmu.edu/general/lmoments)

In [ ]:
from scipy.special import loggamma, digamma

def kappaestimation(R_L_Moment1, R_L_Moment2, R_t_ratio3, R_t_ratio4):
    
    l_mo1, l_mo2 = R_L_Moment1 , R_L_Moment2 

    t3, t4 = R_t_ratio3, R_t_ratio4

    Parameter = [0, 0, 0, 0]

    #EPS,MAXIT CONTROL THE TEST FOR CONVERGENCE OF N-R ITERATION
    #MAXSR IS THE MAX. NO. OF STEPLENGTH REDUCTIONS PER ITERATION
    #HSTART IS THE STARTING VALUE FOR H
    #BIG IS USED TO INITIALIZE THE CRITERION FUNCTION

    Oflexperror = "NO overflow"
    eps = 1./(10**10)
    big = 10
    H_start = 1.001
    Maxit = 20
    Maxsr = 10
    DEL1 = 1.
    DEL2 = 1.
    XG = 10.
    XH = 10.
    
    
    OFLEXP, OFLGAM= 170., 53.

    if l_mo2 <= 0:
        return "L_moments invalid"
    elif np.abs(t3) >= 1 or np.abs(t4) >= 1:
        
        return "L_moments invalid"
    elif t4 <= (5*t3*(t3-1))/4:
        return "L_moments invalid"
    elif t4 >= (5*t3*t3+1)/6.:
        return "Tau 3 and Tau 4 are invalid"



    G = (1 - 3*t3)/(1 + t3)
    H = H_start
    Z = G+H*.725
    x_dist = big

    MAXITLOOPEND = False 

    for i in range(Maxit):
        MAXSRLOOPEND = False
        for j in range(Maxsr):
            
            if G > OFLGAM:
                return print("Overflow ln54")

            if H > 0:
                
                U1 = np.exp(loggamma(1/H) - loggamma(1/H+1+G))
                U2 = np.exp(loggamma(2/H) - loggamma(2/H+1+G))
                U3 = np.exp(loggamma(3/H) - loggamma(3/H+1+G))
                U4 = np.exp(loggamma(4/H) - loggamma(4/H+1+G))
                
            else:
                
                U1 = np.exp(loggamma(-1/H-G) - loggamma(-1/H+1))
                U2 = np.exp(loggamma(-2/H-G) - loggamma(-2/H+1))
                U3 = np.exp(loggamma(-3/H-G) - loggamma(-3/H+1))
                U4 = np.exp(loggamma(-4/H-G) - loggamma(-4/H+1))
                
            l_moment2 = U1 - 2*U2
            l_moment3 = -U1+6*U2-6*U3
            l_moment4 = U1-12*U2+30*U3-20*U4
            
           

            if l_moment2 == 0:
                return "Overflow ln62"

            tau3 = l_moment3/l_moment2
            tau4 = l_moment4/l_moment2
            eps1 = tau3 - t3
            eps2 = tau4 - t4

            DIST = max(np.abs(eps1), np.abs(eps2))

            if DIST >= x_dist:
                DEL1=.5*DEL1
                DEL2=.5*DEL2
                G=XG-DEL1
                H=XH-DEL2
                
            else:
                break


            if j >= Maxsr-1:
                MAXSRLOOPEND = True

        if MAXSRLOOPEND is True:
            return print("TOO MANY STEPLENGTH REDUCTIONS")


        if DIST >= eps:
            XG = G
            XH = H
            XZ = Z
            x_dist = DIST
            RHH = 1/(H*H)
            if (H>0):
                U1G = -U1*digamma(1/H+1+G)
                U2G = -U2*digamma(2/H+1+G)
                U3G = -U3*digamma(3/H+1+G)
                U4G = -U4*digamma(4/H+1+G)
                U1H = RHH*(-U1G-U1*digamma(1/H))
                U2H = 2*RHH*(-U2G-U2*digamma(2/H))
                U3H = 3*RHH*(-U3G-U3*digamma(3/H))
                U4H = 4*RHH*(-U4G-U4*digamma(4/H))
            else:
                U1G = -U1*digamma(-1/H-G)
                U2G = -U2*digamma(-2/H-G)
                U3G = -U3*digamma(-3/H-G)
                U4G = -U4*digamma(-4/H-G)
                U1H = RHH*(-U1G-U1*digamma(-1/H+1))
                U2H = 2*RHH*(-U2G-U2*digamma(-2/H+1))
                U3H = 3*RHH*(-U3G-U3*digamma(-3/H+1))
                U4H = 4*RHH*(-U4G-U4*digamma(-4/H+1))
            DL2G = U1G - 2*U2G
            DL2H = U1H - 2*U2H
            DL3G = -U1G+6*U2G-6*U3G
            DL3H = -U1H + 6*U2H-6*U3H
            DL4G=U1G-12*U2G+30*U3G-20*U4G
            DL4H=U1H-12*U2H+30*U3H-20*U4H
            D11=(DL3G-tau3*DL2G)/l_moment2
            D12=(DL3H-tau3*DL2H)/l_moment2
            D21=(DL4G-tau4*DL2G)/l_moment2
            D22=(DL4H-tau4*DL2H)/l_moment2
            DET=D11*D22-D12*D21
            H11= D22/DET
            H12=-D12/DET
            H21=-D21/DET
            H22= D11/DET
            DEL1=eps1*H11+eps2*H12
            DEL2=eps1*H21+eps2*H22

            G=XG-DEL1
            H=XH-DEL2
            Z=G+H*.725


            FACTOR=1
            if(G<=-1.):
                FACTOR=.8*(XG+1)/DEL1

            if(H<=-1.):
                FACTOR=min(FACTOR,.8*(XH+1)/DEL2)

            if(Z<=-1.):
                FACTOR=min(FACTOR,.8*(XZ+1)/(XZ-Z))

            if(H<=0 and G*H <= -1):
                FACTOR=min(FACTOR,.8*(XG*XH+1)/(XG*XH-G*H))

            if (FACTOR == 0):
                
                continue
            
            else:
                DEL1=DEL1*FACTOR
                DEL2=DEL2*FACTOR
                G=XG-DEL1
                H=XH-DEL2
                Z=G+H*.725

            if i == Maxit-1:
                MAXITLOOPEND == True
            continue
        
        break

        


    if MAXITLOOPEND is True:
        return "END OF NEWTON-RAPHSON ITERATION NOT CONVERGED"


    print("CONVERGED")

    Parameter[0] = H
    Parameter[1] = G
    temp = loggamma(1+G)
    if temp > OFLEXP:
        Oflexperror = print("ITERATION FOR H AND K CONVERGED, BUT OVERFLOW \
        WOULD HAVE OCCURRED WHEN CALCULATING MEAN AND STD")

    GAM = np.exp(temp)
    temp = (1+G)*np.log(np.abs(H))
    if temp > OFLEXP:
        Oflexperror = print("ITERATION FOR H AND K CONVERGED, BUT OVERFLOW \
        WOULD HAVE OCCURRED WHEN CALCULATING MEAN AND STD")

    HH = np.exp(temp)
    Parameter[3]  = l_mo2*G*HH/(l_moment2*GAM)
    Parameter[2] = l_mo1 - Parameter[3]/G*(1-GAM*U1/HH)

    print(Oflexperror)
  
    return np.round(Parameter[0], 3), np.round(Parameter[1], 3), np.round(Parameter[2], 3), np.round(Parameter[3], 3)


#PWR br estimation Fundamental of statistical hydrology (Workes well)

def pwm_br_2(r, sample):
    
    if isinstance(sample, pd.DataFrame):
        sample.sort_values(by="Max Annuel", ascending=True, inplace=True)
        sample = sample["Max Annuel"].values
    else:
        sample = np.sort(sample)
    
    n = len(sample)
    
    br =  np.array([(comb(i - 1, r, exact=True)/comb(n-1, r, exact=True))*sample[i-1] for i in range(1, n+1)]).sum()/n
    return br
                   
def lmoments(l, sample):
    
    if l == 1:
        return pwm_br_2(0, sample)
    elif l==2:
        return 2*pwm_br_2(1, sample) - pwm_br_2(0, sample)
    elif l == 3:
        return 6*pwm_br_2(2, sample) - 6*pwm_br_2(1, sample) + pwm_br_2(0, sample)
    elif l == 4:
        return 20*pwm_br_2(3, sample) - 30*pwm_br_2(2, sample) + 12*pwm_br_2(1, sample) - pwm_br_2(0, sample)
                   
#sample l moments ratios for r >=3
#r = 3 L-Skewness, r = 4 L-Kurtosis

def lmoments_ratios(r, sample):
    return lmoments(r, sample)/lmoments(2, sample)

#sample l-CV (Coefficient-L-Variation)
def l_cv(sample):
    return lmoments(2, sample)/lmoments(1, sample)